In [0]:
import unittest

In [0]:
# Arrange
from datetime import date, timedelta

sum_costs = {
    '2023-8': 3407906.87, 
    '2023-7': 3357881.88, 
    '2023-6': 5320501.56, 
    '2023-5': 3684105.72, 
    '2023-4': 3529240.22, 
    '2023-3': 3671724.15, 
    '2023-2': 3441106.94, 
    '2023-1': 3835808.52, 
    '2022-9': 3500293.32, 
    '2022-8': 3544407.12, 
    '2022-7': 3455565.51, 
    '2022-6': 3272142.0, 
    '2022-5': 2996465.29, 
    '2022-12': 3942270.81, 
    '2022-11': 4026686.53, 
    '2022-10': 3911687.97
}
today = date.today()
last_month = today.replace(day=1) - timedelta(days=1)
rate = 1
raw_azure = "default.raw_azure"
azure_data = "default.azure_data"

In [0]:
# Act
df = spark.sql(f"""
    SELECT Year || '-' || Month AS YearMonth, ROUND(SUM(Cost), 2) AS SumCost FROM {azure_data}
    WHERE Date < '{last_month.year}-{last_month.month}-01'
    GROUP BY YearMonth
""")
dict_costs = {row.YearMonth : row.SumCost for row in df.collect()}

In [0]:
# Assert
class TestAllocationCost(unittest.TestCase):

    def test_sum_cost_equal(self):
        for year_month, sum_cost in sum_costs.items():
            assert year_month in dict_costs, f"{year_month} not found in {azure_data}"
            assert sum_cost == dict_costs[f"{year_month}"], f"{year_month} : {sum_cost} not equal in {azure_data}"

In [0]:
# Run Test 
test_allocation_cost = unittest.TestLoader().loadTestsFromTestCase(TestAllocationCost)
suite = unittest.TestSuite([test_allocation_cost])

assert unittest.TextTestRunner(verbosity=2).run(suite).wasSuccessful(), "Test failed. Refer logs"